In [1]:
import os
import sys
import glob
import shutil
import pandas as pd
from ipywidgets import interact
from copy import deepcopy
import matplotlib.pyplot as plt
print('OK')

OK


In [2]:
sys.path.append('..')

In [3]:
from dataset.dataset.dsindex import FilesIndex
from preprocessing.ct_batch_masked import CTImagesBatchMasked
from preprocessing.ct_batch import CTImagesBatch

### Global index

In [4]:
DIR_LUNA = '/notebooks/data/MRT/luna/s*/*.mhd'
DIR_DUMP = '/notebooks/Koryagin/Batch_dev/dir_for_dumps/'

In [5]:
if os.path.exists(DIR_DUMP):
    shutil.rmtree(DIR_DUMP)

In [6]:
ind = FilesIndex(path=DIR_LUNA, no_ext=True)

### Create batch by subindexing

In [7]:
batch_masked = CTImagesBatchMasked(ind.create_subset(ind.index[[714, 297, 299, 672]]))

### Test actions

* check load

In [8]:
batch_masked.load(fmt='raw')

In [9]:
batch_copy = deepcopy(batch_masked)

* check attrs

In [10]:
batch_masked._data.shape

(1113, 512, 512)

In [11]:
batch_masked._lower_bounds

array([  0, 258, 592, 853])

In [12]:
batch_masked._upper_bounds

array([ 258,  592,  853, 1113])

In [13]:
batch_masked.history

[{'method': 'load', 'params': {}}]

In [14]:
batch_masked.spacing

{'1.3.6.1.4.1.14519.5.2.1.6279.6001.161855583909753609742728521805': array([ 0.78125,  0.78125,  1.     ]),
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.187108608022306504546286626125': array([ 0.68359399,  0.68359399,  1.25      ]),
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.219428004988664846407984058588': array([ 0.859375,  0.859375,  1.25    ]),
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.280072876841890439628529365478': array([ 0.5859375,  0.5859375,  1.       ])}

* check load mask

In [15]:
nodules = pd.read_csv('/notebooks/data/MRT/luna/CSVFILES/annotations.csv')
nodules.head(2)

,seriesuid,coordX,coordY,coordZ,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.783651,-211.925149,-227.121250,4.224708


In [16]:
batch_masked = batch_masked.load_mask(nodules_df=nodules)

In [17]:
len(batch_masked.mask)

4

* check resize, segment

In [18]:
batch_masked = batch_masked.resize(num_slices_new=128, num_x_new=256, num_y_new=256,
                                   num_threads=20).segment(erosion_radius=2, num_threads=20)

* check history

In [19]:
batch_masked.history

[{'method': 'load', 'params': {}},
 {'method': 'resize',
  'params': {'num_slices_new': 128,
   'num_threads': 20,
   'num_x_new': 256,
   'num_y_new': 256,
   'order': 3}},
 {'method': 'segmentation',
  'params': {'erosion_radius': 2, 'num_threads': 20}}]

* check shape of mask

In [20]:
batch_masked.mask._data.shape

(512, 256, 256)

* check dump

In [21]:
batch_masked.dump(DIR_DUMP)

### Build batch from dump and compare

In [22]:
ind_dumped = FilesIndex(path=DIR_DUMP + '/*', dirs=True)

In [23]:
ind_dumped.index

array(['1.3.6.1.4.1.14519.5.2.1.6279.6001.219428004988664846407984058588',
       '1.3.6.1.4.1.14519.5.2.1.6279.6001.187108608022306504546286626125',
       '1.3.6.1.4.1.14519.5.2.1.6279.6001.280072876841890439628529365478',
       '1.3.6.1.4.1.14519.5.2.1.6279.6001.161855583909753609742728521805'], 
      dtype='<U108')

In [24]:
batch_from_dump = CTImagesBatch(ind_dumped)

In [25]:
batch_from_dump.load(fmt='blosc')

In [26]:
def plot_slices(height, num_pers, *batches):
    fig, axes = plt.subplots(1, len(batches), figsize = (10, len(batches)*8))
    com_args = dict(clim=(-1200, 300), cmap=(plt.cm.gray))
    
    for i in range(len(batches)):
        if batches[i][1] == 'scan':
            axes[i].imshow(batches[i][0].get_axial_slice(num_pers, height), **com_args)
        else:
            axes[i].imshow(batches[i][0].get_axial_slice(num_pers, height), cmap=plt.cm.gray)
    plt.show()

In [27]:
batch_copy._lower_bounds.shape[0]

4

In [28]:
batch_masked.indices

array(['1.3.6.1.4.1.14519.5.2.1.6279.6001.219428004988664846407984058588',
       '1.3.6.1.4.1.14519.5.2.1.6279.6001.280072876841890439628529365478',
       '1.3.6.1.4.1.14519.5.2.1.6279.6001.187108608022306504546286626125',
       '1.3.6.1.4.1.14519.5.2.1.6279.6001.161855583909753609742728521805'], 
      dtype='<U101')

In [29]:
batch_from_dump.indices

array(['1.3.6.1.4.1.14519.5.2.1.6279.6001.219428004988664846407984058588',
       '1.3.6.1.4.1.14519.5.2.1.6279.6001.187108608022306504546286626125',
       '1.3.6.1.4.1.14519.5.2.1.6279.6001.280072876841890439628529365478',
       '1.3.6.1.4.1.14519.5.2.1.6279.6001.161855583909753609742728521805'], 
      dtype='<U108')

In [30]:
interact(lambda height: plot_slices(height, batch_masked.indices[1], [batch_copy, 'scan'], [batch_from_dump, 'scan'],
                                    [batch_masked.mask, 'mask']), height=(0.01, 0.99, 0.01))

<function __main__.<lambda>>